# Capability — PPE Detection
Locate protective equipment on an assembly line and highlight every instance with normalized Perceptron geometry.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericpence/perceptron_repo/blob/main/cookbook/recipes/capabilities/object-detection/object-detection.ipynb)

## Install dependencies
Install the SDK plus Pillow so we can preview grounded overlays.

In [ ]:
%pip install --upgrade perceptron pillow --quiet

## Configure the Perceptron client
Authenticate once, then resolve the PPE asset for the remaining cells.

In [ ]:
from pathlib import Path

from IPython.display import Image as IPyImage
from IPython.display import display
from PIL import Image, ImageDraw

from cookbook.utils import cookbook_asset
from perceptron import configure, image, perceive, text

# configure() reads PERCEPTRON_API_KEY from the environment.
configure(
    provider="perceptron",
    # model="isaac-0.1",  # Enable once the SDK supports the model argument.
)

SCENE_PATH = cookbook_asset("capabilities", "detection", "ppe_line.webp")
ANNOTATED_PATH = Path("ppe_line_annotated.png")
if not SCENE_PATH.exists():
    raise FileNotFoundError(f"Missing asset: {SCENE_PATH}")

print(f"Calling Perceptron | input={SCENE_PATH}")

## Build a detection helper
Use the `@perceive` decorator with `expects="box"` so each detection returns normalized bounding boxes.

In [ ]:
TARGET_CLASSES = ["safety helmet", "safety vest"]


@perceive(expects="box", allow_multiple=True)
def detect_ppe(frame_path):
    frame = image(frame_path)
    classes_text = ", ".join(TARGET_CLASSES)
    prompt = text(
        "Find every worker wearing PPE. Focus on helmets and high-visibility vests. "
        "Return one bounding box per instance and include the item name in the mention attribute."
    )
    return frame + prompt

## Run the detection request
Invoke the helper on the PPE line image to retrieve grounded regions.

In [ ]:
detection = detect_ppe(str(SCENE_PATH))
print(detection.text)
boxes = detection.points or []
print(f"Returned {len(boxes)} boxes")

## Render grounded results
Convert the normalized coordinates to pixels and overlay them for quick inspection.

In [ ]:
img = Image.open(SCENE_PATH).convert("RGB")
draw = ImageDraw.Draw(img)


def to_px(point):
    return point.x / 1000 * img.width, point.y / 1000 * img.height


for box in boxes:
    top_left = to_px(box.top_left)
    bottom_right = to_px(box.bottom_right)
    draw.rectangle([top_left, bottom_right], outline="dodgerblue", width=3)
    draw.text(top_left, box.mention or "ppe", fill="dodgerblue")

img.save(ANNOTATED_PATH)
display(IPyImage(filename=str(ANNOTATED_PATH)))
print(f"Saved annotated output to {ANNOTATED_PATH}")

## Conclusion & next steps
- Adjust `TARGET_CLASSES` and the prompt to fit your environment.
- Enable `stream=True` inside `@perceive` for incremental detections.
- Add exemplar shots (see the in-context learning recipe) when classes are ambiguous.